In [ ]:
!pip install pandas transformers guidance numpy evaluate rouge-score nltk py-rouge accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.6/223.6 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.6/311.6 kB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 50.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.9/91.9 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 41.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.9 MB

In [ ]:
import pandas as pd
from sklearn.metrics import f1_score, accuracy_score
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch
import json
import nltk
import numpy as np
from guidance import gen
from evaluate import load
import csv
import nltk.translate.bleu_score as bleu
from rouge import Rouge
from transformers import BertTokenizer, BertModel
import torch
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
import warnings
from typing import Any, Dict, Tuple
import re
import os

warnings.filterwarnings("ignore", category=UserWarning, message="A new version of the following files was downloaded from")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


# Load Dataset

- id: unique identifier of the example
- source: original model input for paraphrase generation
- hyp1: first alternative paraphrase of the source
- hyp2: second alternative paraphrase of the source
- label: hyp1 or hyp2, based on which of those has been annotated as hallucination
- type: hallucination category assigned.
  - Possible values: addition, named-entity, number, conversion, date, tense, negation, gender, pronoun, antonym, natural

## Trial dataset

This is a small list of examples, provided to help the participants get familiar with the task. Each example contains the following fields:
- id
- source
- hyp1
- hyp2
- label
- type

In [ ]:
from datasets import load_dataset

#load the trial data for both English and Swedish
trial_ds = load_dataset("Eloquent/HalluciGen-PG", name="trial")

#load the trial data only for Swedish
trial_ds_sv = load_dataset("Eloquent/HalluciGen-PG", name="trial", split="trial_swedish")

print(trial_ds.keys())

trial_df_en = trial_ds['trial_english'].to_pandas()
trial_df_se= trial_ds['trial_swedish'].to_pandas()

print('English:')

display(trial_df_en.head())

print('Swedish')
display(trial_df_se.head())

Generating trial_swedish split:   0%|          | 0/20 [00:00<?, ? examples/s]

Generating trial_english split:   0%|          | 0/16 [00:00<?, ? examples/s]

dict_keys(['trial_swedish', 'trial_english'])
English:


,id,source,type,hyp1,hyp2,label
0,0,The population has declined in some 210 of the...,antonym,In the majority of Sweden's 280 municipalities...,In the majority of Sweden's 280 municipalities...,hyp1
1,1,The draft agenda as drawn up by the Conference...,negation,The Conference of Presidents hasn't distribute...,The Conference of Presidents has distributed t...,hyp1
2,2,I am always grateful for comments and suggesti...,antonym,I think I need to listen to everyone's views i...,I think I need to listen to everyone's views i...,hyp1
3,3,The fact is that a key omission from the propo...,named entity,Agenda 2030 does not include a chapter on rene...,Agenda 2000 does not include a chapter on rene...,hyp1
4,4,"We need quite specific legislative proposals, ...",natural,Legislative proposals based on the Treaty of A...,Legislative proposals based solely on the Trea...,hyp2


Swedish


,id,source,type,hyp1,hyp2,label
0,0,"Men intäkterna från mjukvarulicenser, ett mått...",number,"Intäkter från programvarulicenser, en metrik s...","Intäkter från programvarulicenser, en metrik s...",hyp2
1,1,Hongkong-universitet samarbetar med universite...,natural,University of Hong Kong samarbetar med olika k...,University of Hong Kong samarbetar med olika k...,hyp1
2,2,Län med befolkningsminskning kommer att vara V...,named entity,"Vermillion, Posey och Madison är län som komme...","Vermillion, Posey och Marion är län som kommer...",hyp2
3,3,Google presenterar en prototyp för en självkör...,addition,En prototyp för en självkörande bil presentera...,Enligt ett blogginlägg från Google har företag...,hyp2
4,4,Lagförslaget säger att en kvinna som genomgår ...,negation,Förslaget stadgar att en kvinna som genomgår e...,Förslaget stadgar att en kvinna som genomgår e...,hyp1


In [ ]:
print("Source:", trial_df_en.loc[0, 'source'])
print("Hyp1:", trial_df_en.loc[0, 'hyp1'])
print("Hyp2:", trial_df_en.loc[0, 'hyp2'])


Source: The population has declined in some 210 of the 280 municipalities in Sweden, mainly in inland central and northern Sweden.
Hyp1: In the majority of Sweden's 280 municipalities, the population has gone up.
Hyp2: In the majority of Sweden's 280 municipalities, the population has gone down.


In [ ]:
#load the test data for the generation step in both English and Swedish
data = load_dataset("Eloquent/HalluciGen-PG", "test_generation")


print(data.keys())

data_df_en = data['test_generation_english'].to_pandas()
data_df_se= data['test_generation_swedish'].to_pandas()


Generating test_generation_swedish split:   0%|          | 0/76 [00:00<?, ? examples/s]

Generating test_generation_english split:   0%|          | 0/119 [00:00<?, ? examples/s]

dict_keys(['test_generation_swedish', 'test_generation_english'])


In [ ]:
data_df_en

,id,source
0,0,It has enabled us to support and encourage an ...
1,1,"Therefore, I am calling for an increase in the..."
2,2,"In other words, a person may be prevented from..."
3,3,None of my 34 amendments were adopted and my a...
4,4,The final key feature of the report is its ori...
...,...,...
114,114,I am referring to the current vote but in rela...
115,115,When I became a Member of the European Parliam...
116,116,"Mr President, let me express my thanks for the..."
117,117,We can replace 16 out of 626 Members of the Eu...


In [ ]:
data_df_se

,id,source
0,0,Kvinnor kommer att möta högre bilförsäkringspr...
1,1,"Rörelseintäkterna var 1,45 miljarder dollar, e..."
2,2,"Mandela tillbaka på sjukhus i ""allvarligt men ..."
3,3,Egypten fryser Muslimska brödraskapets tillgån...
4,4,"P-pillret, som de kallar ""polypiller"", skulle ..."
...,...,...
71,71,Du kommer att komma på saker! Jag kommer att b...
72,72,Påven uppmanar till åtgärder mot klimatförändr...
73,73,Egyptens ledare inför undantagstillstånd i tre...
74,74,En svartvit bild av en katt som ligger på en m...


In [ ]:
from google.colab import userdata
api_key = userdata.get('OPENAI_API_KEY')

In [ ]:
os.environ['OPENAI_API_KEY'] = api_key

In [ ]:
from guidance import models, instruction, system, user, assistant
gpt = models.OpenAI("gpt-3.5-turbo")

# Prompt Version 1

In [1]:
def generate_paraphrase(source):
    # Generate the prompt

    answer_format = {"hyp+": " ", "hyp-": " "}

    user_prompt = f'''
    Given the src below, generate a paraphrase hypothesis  hyp+ that is supported by src and a second paraphrase hyp- that is not supported by src.

    Provide the result in the following format: {answer_format}

    Src: {source}

    Result:
    '''

    with system():
        lm = gpt + "You are a text generator. You are specialized in paraphrasing texts."

    with user():
        lm += user_prompt

    with assistant():
        lm += gen("answer")

    result = lm["answer"]

    print(result)

    return result

In [ ]:
def generate_paraphrase_se(source):
    # Generate the prompt

    answer_format = {"hyp+": " ", "hyp-": " "}

    user_prompt = f'''
    Med tanke på källan nedan, generera en parafras-hypotes hyp+ som stöds av källan och en andra parafras hyp- som inte stöds av källan.

    Ge resultatet i följande format: {answer_format}

    Källa: {source}

    Resultat:
    '''

    with system():
        lm = gpt + "Du är en textgenerator. Du är specialiserad på att parafrasera texter"

    with user():
        lm += user_prompt

    with assistant():
        lm += gen("answer")

    result = lm["answer"]

    print(result)

    return result

In [ ]:
import json
import re

def extract_json_data(result_text):
    # Find the start and end indices of the JSON data
    start_index = result_text.find('{')
    end_index = result_text.find('}') + 1  # Add 1 to include the closing brace

    # Extract the JSON data from the string
    result_text = result_text[start_index:end_index]

    # Replace single quotes used as delimiters with double quotes

    result_text = result_text.replace("{'", '{"').replace("': '", '": "').replace(".'", '."').replace("', '", '", "').replace(", '", ', "').replace("',", '",').replace("\n'", '"').replace("\\'", "'").replace("'}", '"}')

    print("Formatted JSON String: ", result_text)

    # Convert the JSON data to a Python dictionary
    data_dict = json.loads(result_text)

    return data_dict

In [ ]:
source = "We voted for Amendments Nos 22 to 25, even though they do not take adequate measures to prevent these problems from happening."
result = generate_paraphrase(source)
extract_json_data(result)

{'hyp+': 'We supported Amendments Nos 22 to 25 despite their lack of sufficient measures to address these issues.', 'hyp-': 'We opposed Amendments Nos 22 to 25 due to their failure to address these issues adequately.'}
Formatted JSON String:  {"hyp+": "We supported Amendments Nos 22 to 25 despite their lack of sufficient measures to address these issues.", "hyp-": "We opposed Amendments Nos 22 to 25 due to their failure to address these issues adequately."}


{'hyp+': 'We supported Amendments Nos 22 to 25 despite their lack of sufficient measures to address these issues.',
 'hyp-': 'We opposed Amendments Nos 22 to 25 due to their failure to address these issues adequately.'}

In [ ]:
result = "Gen text:  {'hyp+': 'The PSE Group has been actively advocating against the death penalty in the United States, with a specific focus on the cases of Greg Summers and Leonard Peltier.', 'hyp-': 'The European Union has never condemned the death penalty in the United States, and Mrs. d\'Ancona has never spoken out against it.'}"
#result2 = '{"hyp+": "As a member of the Socialist Group, I am pleased to announce our wholehearted endorsement of the proposal', "hyp-": "The Socialist Group has always been opposed to this proposal and will continue to resist its implementation"}'

extracted_hyp = extract_json_data(result)

Formatted JSON String:  {"hyp+": "The PSE Group has been actively advocating against the death penalty in the United States, with a specific focus on the cases of Greg Summers and Leonard Peltier.", "hyp-": "The European Union has never condemned the death penalty in the United States, and Mrs. d'Ancona has never spoken out against it."}


## Generate dataset

In [ ]:

import os
import numpy as np
import pandas as pd

def save_results(generate_paraphrase_func, prompt, df, csv_filename):

    # Check if the CSV file exists
    if os.path.exists(csv_filename):
        # Read the CSV file using pandas
        existing_csv = pd.read_csv(csv_filename)

        # Check if the DataFrame is not empty
        if not existing_csv.empty:
            # Get the highest ID from the DataFrame
            next_id = existing_csv['id'].max() + 1

            remainder_df = df[df['id'] >= next_id]

            chunks = np.array_split(remainder_df, len(remainder_df) // 10 if len(remainder_df) % 10 == 0 else len(remainder_df) // 10 + 1)
    else:
        # If the CSV file doesn't exist, set highest_id to 1
        next_id = 0

        chunks = np.array_split(df, len(df) // 10 if len(df) % 10 == 0 else len(df) // 10 + 1)

    print("Highest ID: ", next_id)
    # Split DataFrame into chunks of 10 rows

    for chunk in chunks:
        # Iterate through each row in the chunk
        for index, row in chunk.iterrows():
            source = row['source']

            try:
              # Generate paraphrase
              result = generate_paraphrase_func(source)
              extracted_hyp = extract_json_data(result)

              hyp_support = extracted_hyp['hyp+']
              print("Hyp+ : ", hyp_support)
              hyp_hallu = extracted_hyp['hyp-']
              print("Hyp- : ", hyp_hallu)

              print('source: ',source)

              # Update DataFrame with new values
              chunk.at[index, 'hyp+'] = hyp_support
              chunk.at[index, 'hyp-'] = hyp_hallu
              chunk.at[index, 'prompt'] = prompt
            except:
                print(f"Skipping row {index} due to error in data extraction.")
                continue

        # Append chunk to CSV file
        if not os.path.isfile(csv_filename):
            chunk.to_csv(csv_filename, mode='w', header=True, index=False)
        else:
            chunk.to_csv(csv_filename, mode='a', header=False, index=False)

        # Load CSV file
        df = pd.read_csv(csv_filename)
        display(df)

In [ ]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group2_results/generation_task/generation_task_gpt35/results_gpt35_en.csv"

prompt = '''
    Given the src below, generate a paraphrase hypothesis  hyp+ that is supported by src and a second paraphrase hyp- that is not supported by src.

    Provide the result in the following format: {answer_format}

    Src: {source}

    Result:
'''

save_results(generate_paraphrase, prompt, data_df_en, csv_filename)

{'hyp+': 'The Council is erring by solely relying on the United States to come up with solutions for the Middle East issue.', 'hyp-': 'The Council is correct in entrusting the United States with the responsibility of formulating proposals for the Middle East problem.'}
Formatted JSON String:  {"hyp+": "The Council is erring by solely relying on the United States to come up with solutions for the Middle East issue.", "hyp-": "The Council is correct in entrusting the United States with the responsibility of formulating proposals for the Middle East problem."}
Hyp+ :  The Council is erring by solely relying on the United States to come up with solutions for the Middle East issue.
Hyp- :  The Council is correct in entrusting the United States with the responsibility of formulating proposals for the Middle East problem.
source:  The Council is making a mistake by leaving it entirely up to the United States to formulate proposals for a solution to the Middle East problem.


,id,source,hyp+,hyp-,prompt
0,0,It has enabled us to support and encourage an ...,It has allowed us to back and promote a sharin...,It has made it difficult for us to provide sup...,"\n Given the src below, generate a paraphra..."
1,1,"Therefore, I am calling for an increase in the...","As a result, I am proposing a rise in the pric...","Consequently, I am suggesting a decrease in th...","\n Given the src below, generate a paraphra..."
2,2,"In other words, a person may be prevented from...","Put differently, an individual might be stoppe...","Conversely, a person might not be allowed to a...","\n Given the src below, generate a paraphra..."
3,3,None of my 34 amendments were adopted and my a...,None of the 34 changes I suggested were accept...,My suggestions for amendments were all rejecte...,"\n Given the src below, generate a paraphra..."
4,4,The final key feature of the report is its ori...,The report's ultimate crucial aspect is its fo...,The report's main characteristic is its layout...,"\n Given the src below, generate a paraphra..."
...,...,...,...,...,...
114,114,I am referring to the current vote but in rela...,I am talking about the present vote in connect...,I am discussing the current ballot in regards ...,"\n Given the src below, generate a paraphra..."
115,115,When I became a Member of the European Parliam...,"Upon joining the European Parliament in 1994, ...","Upon entering the European Parliament in 1994,...","\n Given the src below, generate a paraphra..."
116,116,"Mr President, let me express my thanks for the...",I want to thank the President for his earlier ...,"The President's remarks were not appreciated, ...","\n Given the src below, generate a paraphra..."
117,117,We can replace 16 out of 626 Members of the Eu...,While 16 out of 626 Members of the European Pa...,Only 16 out of 626 Members of the European Par...,"\n Given the src below, generate a paraphra..."


In [ ]:
# update failed rows
def process_csv(file_path, generate_paraphrase_func, prompt):
    # Read the CSV file into a pandas DataFrame
    df = pd.read_csv(file_path)

    # Initialize an empty list to store IDs
    missing_ids = []

    # Iterate over each row
    for index, row in df.iterrows():
        # Check if any of the specified columns are empty
        if pd.isna(row['prompt']) and pd.isna(row['hyp+']) and pd.isna(row['hyp-']):
            # Update the "source" and "hyp+" columns with the specified text
            source = row['source']

            missing_ids.append(row['id'])
            try:
              # Generate paraphrase
              result = generate_paraphrase_func(source)
              extracted_hyp = extract_json_data(result)

              hyp_support = extracted_hyp['hyp+']
              print("Hyp+ : ", hyp_support)
              hyp_hallu = extracted_hyp['hyp-']
              print("Hyp- : ", hyp_hallu)

              print('source: ',source)

              # Update DataFrame with new values
              df.at[index, 'hyp+'] = hyp_support
              df.at[index, 'hyp-'] = hyp_hallu
              df.at[index, 'prompt'] = prompt

              print("ADDED")
            except:
                print(f"Skipping row {index} due to error in data extraction.")
                continue

    rows_with_missing_ids = df[df['id'].isin(missing_ids)]

    display(rows_with_missing_ids)

    df.to_csv(file_path, header=True, index=False)

In [ ]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group2_results/generation_task/generation_task_gpt35/results_gpt35_en.csv"

prompt = '''
    Given the src below, generate a paraphrase hypothesis  hyp+ that is supported by src and a second paraphrase hyp- that is not supported by src.

    Provide the result in the following format: {answer_format}

    Src: {source}

    Result:
'''

result = process_csv(csv_filename, generate_paraphrase, prompt)
result

,id,source,hyp+,hyp-,prompt


In [ ]:
# save a csv file in the task format

import pandas as pd

# Load the CSV file
df = pd.read_csv('/content/drive/MyDrive/Master Project-Hallucination/Group2_results/generation_task/generation_task_gpt35/results_gpt35_en.csv')

# Select only the columns 'id', 'hyp+', and 'hyp-'
df = df[['id', 'hyp+', 'hyp-']]

# Save the new DataFrame to a CSV file
df.to_csv('/content/drive/MyDrive/Master Project-Hallucination/Group2_results/final_gpt35_en.csv', index=False)

# Swedish

In [ ]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group2_results/generation_task/generation_task_gpt35/results_gpt35_se.csv"

prompt = '''
    användare
    Med tanke på källan nedan, generera en parafras-hypotes hyp+ som stöds av källan och en andra parafras hyp- som inte stöds av källan.
    Ge resultatet i följande format: {answer_format}
    Källa: {source}
    Resultat:
      '''

save_results(generate_paraphrase_se, prompt, data_df_se, csv_filename)

{'hyp+': 'Invånare i Texas som kämpar med höga kostnader för hemförsäkringar kan äntligen se fram emot en minskning av sina premier.', 'hyp-': 'Invånare i Texas har inte märkt någon förändring i sina hemförsäkringspremier trots de höga kostnaderna.'}
Formatted JSON String:  {"hyp+": "Invånare i Texas som kämpar med höga kostnader för hemförsäkringar kan äntligen se fram emot en minskning av sina premier.", "hyp-": "Invånare i Texas har inte märkt någon förändring i sina hemförsäkringspremier trots de höga kostnaderna."}
Hyp+ :  Invånare i Texas som kämpar med höga kostnader för hemförsäkringar kan äntligen se fram emot en minskning av sina premier.
Hyp- :  Invånare i Texas har inte märkt någon förändring i sina hemförsäkringspremier trots de höga kostnaderna.
source:  Texaner som dignar under skyhöga hemförsäkringspremier kan äntligen få lättnad.


,id,source,hyp+,hyp-,prompt
0,0,Kvinnor kommer att möta högre bilförsäkringspr...,Det förväntas att kvinnor kommer att bli debit...,Det förväntas att män kommer att möta lägre bi...,\n användare\n Med tanke på källan nedan...
1,1,"Rörelseintäkterna var 1,45 miljarder dollar, e...","Företagets intäkter ökade till 1,45 miljarder ...","Företagets intäkter minskade till 1,45 miljard...",\n användare\n Med tanke på källan nedan...
2,2,"Mandela tillbaka på sjukhus i ""allvarligt men ...",Mandela återinlagd på sjukhus i tillstånd som ...,Mandela har återhämtat sig och är tillbaka på ...,\n användare\n Med tanke på källan nedan...
3,3,Egypten fryser Muslimska brödraskapets tillgån...,Egypten har beslutat att frysa tillgångarna fö...,Egypten har beslutat att öka tillgångarna för ...,\n användare\n Med tanke på källan nedan...
4,4,"P-pillret, som de kallar ""polypiller"", skulle ...",NaN,NaN,NaN
...,...,...,...,...,...
71,71,Du kommer att komma på saker! Jag kommer att b...,Det är möjligt att nya idéer kommer att uppstå...,Jag kommer aldrig att kunna komma på något nyt...,\n användare\n Med tanke på källan nedan...
72,72,Påven uppmanar till åtgärder mot klimatförändr...,Påven betonar vikten av att vidta åtgärder för...,Påven förespråkar att inte göra någonting för ...,\n användare\n Med tanke på källan nedan...
73,73,Egyptens ledare inför undantagstillstånd i tre...,Egyptens regeringschef har beslutat att införa...,Egyptens president har beslutat att införa und...,\n användare\n Med tanke på källan nedan...
74,74,En svartvit bild av en katt som ligger på en m...,En bild av en katt som vilar på en matta i sva...,En färgglad bild av en hund som springer på gr...,\n användare\n Med tanke på källan nedan...


In [ ]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group2_results/generation_task/generation_task_gpt35/results_gpt35_se.csv"

prompt = '''
    användare
    Med tanke på källan nedan, generera en parafras-hypotes hyp+ som stöds av källan och en andra parafras hyp- som inte stöds av källan.
    Ge resultatet i följande format: {answer_format}
    Källa: {source}
    Resultat:
      '''

result = process_csv(csv_filename, generate_paraphrase_se, prompt)
result

{'hyp+': 'En kombinationspiller, även kallat "polypiller", skulle kunna inkludera acetylsalicylsyra, ett läkemedel för att sänka kolesterol, tre olika blodtryckssänkande läkemedel i lägre doser än vanligt samt folsyra.', 'hyp-': 'Det nya polypillret innehåller endast acetylsalicylsyra och folsyra utan några blodtryckssänkande läkemedel.'}
Formatted JSON String:  {"hyp+": "En kombinationspiller, även kallat "polypiller", skulle kunna inkludera acetylsalicylsyra, ett läkemedel för att sänka kolesterol, tre olika blodtryckssänkande läkemedel i lägre doser än vanligt samt folsyra.", "hyp-": "Det nya polypillret innehåller endast acetylsalicylsyra och folsyra utan några blodtryckssänkande läkemedel."}
Skipping row 4 due to error in data extraction.


,id,source,hyp+,hyp-,prompt
4,4,"P-pillret, som de kallar ""polypiller"", skulle ...",NaN,NaN,NaN


In [ ]:
# Load the CSV file
df = pd.read_csv("/content/drive/MyDrive/Master Project-Hallucination/Group2_results/generation_task/generation_task_gpt35/results_gpt35_se.csv")

# Select only the columns 'id', 'hyp+', and 'hyp-'
df = df[['id', 'hyp+', 'hyp-']]

# Save the new DataFrame to a CSV file
df.to_csv('/content/drive/MyDrive/Master Project-Hallucination/Group2_results/final_gpt35_se.csv', index=False)